# Visualizing many distributions at once using boxplots and ridgeline plots

This is the fifth installment in a series of blog posts where we reproduce plots from Claus Wilke’s book, *Fundamentals of Data Visualization.* 

This notebook demonstrates how to recreate the boxplots and ridgeline plots found in the “[visualizing many distributions at once](https://clauswilke.com/dataviz/boxplots-violins.html)” chapter of the book.

We will use the `vbar()`, `scatter()`, and `patch()` glyphs to recreate the plots.

In [ ]:
from bokeh.io import output_notebook
import pandas as pd

In [ ]:
output_notebook()

### Figure 9.3

In [ ]:
file = "../data/csv_files/lincoln.csv"
df = pd.read_csv(file)

df["DATE"] = pd.to_datetime(df["DATE"])
df["TAVG"] = (df["TMAX"] + df["TMIN"]) / 2
df["MONTH"] = df.DATE.dt.strftime("%b")

df = df[
    [
        "MONTH",
        "TMIN",
        "TMAX",
        "TAVG",
    ]
]

In [ ]:
qs = df.groupby("MONTH").TAVG.quantile([0.25, 0.5, 0.75]).unstack().reset_index()
qs.columns = ["MONTH", "Q1", "Q2", "Q3"]

iqr = qs.Q3 - qs.Q1
qs["upper"] = qs.Q3 + 1.5 * iqr
qs["lower"] = qs.Q1 - 1.5 * iqr
df = pd.merge(df, qs, on="MONTH", how="left")

df

In [ ]:
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show

In [ ]:
p = figure(
    title="Figure 9.3",
    x_range=df.MONTH.unique(),
    toolbar_location=None,
    height=400,
    width=600,
    x_axis_label="month",
    y_axis_label="mean temperature (F)",
)

source = ColumnDataSource(df)

whisker = Whisker(base="MONTH", upper="upper", lower="lower", source=source)
whisker.upper_head.size = whisker.lower_head.size = 20
p.add_layout(whisker)

p.vbar(
    x="MONTH",
    top="Q2",
    bottom="Q1",
    width=0.8,
    source=source,
    color="#E0E0E0",
    line_color="black",
)

p.vbar(
    x="MONTH",
    top="Q3",
    bottom="Q2",
    width=0.8,
    source=source,
    color="#E0E0E0",
    line_color="black",
)

outliers = df[~df.TAVG.between(df.lower, df.upper)]
p.scatter("MONTH", "TAVG", source=outliers, size=5, color="black")

p.y_range.start = -10
p.yaxis.ticker = [0, 25, 50, 75]
p.grid.grid_line_color = None

show(p)

### Figure 9.8

In [ ]:
import numpy as np

values = []
cats = list(df.MONTH.unique())
for cat in cats:
    month = df[df["MONTH"] == cat]
    m_values = month.TAVG.values
    m_values = np.nan_to_num(m_values, nan=0)
    values.append(m_values)

In [ ]:
from sklearn.neighbors import KernelDensity

positions = np.linspace(-1, 90, 500)

In [ ]:
from bokeh.transform import jitter

In [ ]:
source = ColumnDataSource(df)

p = figure(
    title="Figure 9.8",
    x_range=df.MONTH.unique(),
    toolbar_location=None,
    height=400,
    width=500,
    x_axis_label="month",
    y_axis_label="mean temperature (F)",
)


def ridge(category, data, scale=5):
    return list(zip([category] * len(data), scale * data))


for cat, value in zip(cats, values):
    kde = KernelDensity(kernel="gaussian", bandwidth=3).fit(value[:, np.newaxis])
    log_dens = kde.score_samples(positions[:, np.newaxis])
    x1 = ridge(cat, np.exp(log_dens))
    x2 = ridge(cat, -(np.exp(log_dens)))
    p.harea(x1=x1, y=positions, x2=x2, alpha=0.8, color="#E0E0E0")

p.scatter(
    x=jitter("MONTH", width=0.4, range=p.x_range),
    y="TAVG",
    source=source,
    color="black",
)

p.y_range.start = -10
p.yaxis.ticker = [0, 25, 50, 75]
p.grid.grid_line_color = None

show(p)

### figure 9.12

In [ ]:
file = "../data/csv_files/dw_nominate_house.csv"

df = pd.read_csv(file)
df["year"] = (df.congress) * 2 + 1787

year = df["year"] >= 1963
parties = (df["party_code"] == 100) | (df["party_code"] == 200)
dn = df["cd"] != 0

df = df[year & parties & dn].reset_index(drop=True)

df

In [ ]:
dem = df[df["party_code"] == 100]
rep = df[df["party_code"] == 200]

dems = dem[["year", "dim_1"]]

grouped = dems.groupby("year")["dim_1"]


def ridge(category, data, scale=5):
    return list(zip([category] * len(data), scale * data))


kde_results = []
for year, value in grouped:
    positions = np.linspace(-1.5, 1.5, 1000)
    values = value.values
    kde = KernelDensity(kernel="gaussian", bandwidth=0.3).fit(values[:, np.newaxis])
    log_dens = kde.score_samples(positions[:, np.newaxis])
    kde_results.append({"year": year, "x_range": positions, "log_density": log_dens})

y_range = [year for year, value in grouped]
p = figure(title="Figure 9.12", x_axis_label="DW-NOMINATE score")
# y_range=y_range)

for i, kde_result in enumerate(kde_results):
    density = np.exp(kde_result["log_density"])
    y = ridge(year, density)
    p.patch(
        x=positions, y=density, line_color="blue", fill_color="blue", fill_alpha=0.7
    )

show(p)